In [1]:
import pandas as pd
import numpy as np

In [2]:
results_session = pd.read_csv("./da_cross_session.csv")

### Cross Session

In [3]:
results_session

,Unnamed: 0,align,no_align,time,distance,n_proj,n_epochs,seed,subject,target_subject,multifreq,reg,cross_subject
0,0,0.843416,0.822064,43.299664,les,500,500,2473,1,0,False,1.0,False
1,1,0.846975,0.822064,11.988925,les,500,500,2473,1,0,False,10.0,False
2,2,0.843416,0.822064,43.299664,les,500,500,2473,1,0,False,1.0,False
3,3,0.846975,0.822064,11.988925,les,500,500,2473,1,0,False,10.0,False
4,4,0.843416,0.822064,43.299664,les,500,500,2473,1,0,False,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,0.765152,0.757576,4.147713,logsw,500,500,929,9,0,False,1.0,False
996,996,0.765152,0.757576,4.147713,logsw,500,500,929,9,0,False,1.0,False
997,997,0.765152,0.757576,4.147713,logsw,500,500,929,9,0,False,1.0,False
998,998,0.765152,0.757576,4.147713,logsw,500,500,929,9,0,False,1.0,False


In [4]:
distances = results_session["distance"].unique()
subjects = results_session["subject"].unique()

In [5]:
L_session = {}
for distance in distances:
    L_session[distance] = {}
L_session["no_align"] = {}


reg = 10.0

for distance in distances:
    for s in subjects:
        dico_distances = {"mean_score": 0, "std_score": 0, "mean_time": 0, "std_time":0}
        
        if distance == "les":
            filt = (results_session["distance"] == distance) & (results_session["subject"] == s) & (results_session["reg"] == reg)
            
        else:
            filt = (results_session["distance"] == distance) & (results_session["subject"] == s)
            
        scores = results_session[filt]["align"]
        m_score = scores.mean()
        std_score = scores.std()
        
        times = results_session[filt]["time"]
        m_t = times.mean()
        std_t = times.std()
        
        dico_distances["mean_score"] = m_score
        dico_distances["std_score"] = std_score
        dico_distances["mean_time"] = m_t
        dico_distances["std_time"] = std_t
        
        L_session[distance][s] = dico_distances
        
        
for s in subjects:
    dico_distances = {"mean_score": 0, "std_score": 0, "mean_time": 0, "std_time":0}

    scores = results_session[
        (results_session["distance"] == distance)
        & (results_session["subject"] == s)
    ]["no_align"]
    m_score = scores.mean()
    std_score = scores.std()

    dico_distances["mean_score"] = m_score
    dico_distances["std_score"] = std_score

    L_session["no_align"][s] = dico_distances

In [7]:
acc_session = np.zeros((len(subjects), len(distances)+1))
acc_std_session = np.zeros((len(subjects), len(distances)))
t_session = np.zeros((len(subjects), len(distances)))
t_std_session = np.zeros((len(subjects), len(distances)))

for i, distance in enumerate(distances):
    for j, s in enumerate(subjects):
        acc_session[j, i] = L_session[distance][s]["mean_score"]
        acc_std_session[j, i] = L_session[distance][s]["std_score"]
        t_session[j, i] = L_session[distance][s]["mean_time"]
        t_std_session[j, i] = L_session[distance][s]["std_time"]
        
distance = "no_align"        
for j, s in enumerate(subjects):
    acc_session[j, -1] = L_session[distance][s]["mean_score"]

In [8]:
pd.DataFrame(acc_session, index=subjects, columns=np.concatenate([distances, ["no_align"]]))

,les,lew,spdsw,logsw,no_align
1,0.846975,0.843416,0.846975,0.844840,0.822064
3,0.860806,0.857143,0.855678,0.841026,0.798535
7,0.812274,0.812274,0.810108,0.763177,0.722022
8,0.830258,0.822878,0.835424,0.810332,0.793358
9,0.776515,0.776515,0.773485,0.778788,0.757576


In [9]:
pd.DataFrame(acc_session, index=subjects, columns=np.concatenate([distances, ["no_align"]])).mean(axis=0)

les         0.825366
lew         0.822445
spdsw       0.824334
logsw       0.807632
no_align    0.778711
dtype: float64

In [10]:
pd.DataFrame(acc_session, index=subjects, columns=np.concatenate([distances, ["no_align"]])).std(axis=0)

les         0.032805
lew         0.031052
spdsw       0.033189
logsw       0.036452
no_align    0.039203
dtype: float64

In [11]:
pd.DataFrame(acc_std_session, index=subjects, columns=distances)

,les,lew,spdsw,logsw
1,1.133117e-16,4.485978e-16,0.006015,0.005841
3,3.399350e-16,4.485978e-16,0.003773,0.009818
7,4.532467e-16,1.121495e-16,0.009110,0.013761
8,0.000000e+00,5.607473e-16,0.010437,0.015744
9,3.399350e-16,1.121495e-16,0.006583,0.007879


In [12]:
pd.DataFrame(t_session, index=subjects, columns=distances)

,les,lew,spdsw,logsw
1,11.083333,11.547856,4.427410,4.415610
3,10.978358,11.397449,4.387917,4.361278
7,10.680254,11.876247,4.415600,4.394663
8,10.796004,11.342616,4.354783,4.341887
9,16.676332,10.199346,4.166109,4.163462


In [13]:
pd.DataFrame(t_session, index=subjects, columns=distances).mean(axis=0)

les      12.042856
lew      11.272703
spdsw     4.350364
logsw     4.335380
dtype: float64

In [14]:
pd.DataFrame(t_session, index=subjects, columns=distances).std(axis=0)

les      2.594912
lew      0.634925
spdsw    0.106738
logsw    0.100275
dtype: float64

In [15]:
pd.DataFrame(t_std_session, index=subjects, columns=distances)

,les,lew,spdsw,logsw
1,0.514395,0.023272,0.025845,0.030044
3,0.302465,0.040088,0.017107,0.024172
7,0.044460,0.023522,0.022676,0.026086
8,0.476104,0.034896,0.020857,0.024802
9,0.138757,0.053147,0.014280,0.023982


### Cross Subjects

In [16]:
results_subject = pd.read_csv("./da_cross_subject.csv")

In [17]:
results_subject

,Unnamed: 0,align,no_align,time,distance,n_proj,n_epochs,seed,subject,target_subject,multifreq,reg,cross_subject
0,0,1.000000,1.000000,0.000000,les,500,500,2473,1,1,False,10.0,True
1,1,0.688889,0.522222,14.565961,les,500,500,2473,1,3,False,10.0,True
2,2,0.597786,0.505535,11.734443,les,500,500,2473,1,7,False,10.0,True
3,3,0.689394,0.390152,11.258501,les,500,500,2473,1,8,False,10.0,True
4,4,0.569620,0.265823,10.770443,les,500,500,2473,1,9,False,10.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,0.454212,0.267399,4.226686,logsw,500,500,929,9,1,False,1.0,True
496,496,0.466667,0.285185,4.170060,logsw,500,500,929,9,3,False,1.0,True
497,497,0.324723,0.247232,4.199682,logsw,500,500,929,9,7,False,1.0,True
498,498,0.541667,0.397727,4.149868,logsw,500,500,929,9,8,False,1.0,True


In [18]:
distances = results_subject["distance"].unique()
subjects_src = results_subject["subject"].unique()
subjects_tgt = results_subject["target_subject"].unique()

In [19]:
L_subject = {}
for distance in distances:
    L_subject[distance] = {}
L_subject["no_align"] = {}


for distance in distances:
    for s1 in subjects_src:
        L_subject[distance][s1] = {}
        for s2 in subjects_tgt:
            if s1 != s2:
                L_subject[distance][s1][s2] = {}
        
                scores = results_subject[
                    (results_subject["distance"] == distance)
                    & (results_subject["subject"] == s1)
                    & (results_subject["target_subject"] == s2)
                ]["align"]
                m_score = scores.mean()
                std_score = scores.std()
                
                times = results_subject[
                    (results_subject["distance"] == distance)
                    & (results_subject["subject"] == s1)
                    & (results_subject["target_subject"] == s2)
                ]["time"]
                
                m_t = times.mean()
                std_t = times.std()
                
                L_subject[distance][s1][s2]["mean_score"] = m_score
                L_subject[distance][s1][s2]["std_score"] = std_score
                L_subject[distance][s1][s2]["mean_time"] = m_t
                L_subject[distance][s1][s2]["std_t"] = std_t
        
        
for s1 in subjects_src:
    L_subject["no_align"][s1] = {}
    for s2 in subjects_tgt:
        if s1 != s2:
            dico_distances = {"mean_score": 0, "std_score": 0, "mean_time": 0, "std_time":0}

            scores = results_subject[
                (results_subject["distance"] == distance)
                & (results_subject["subject"] == s1)
                & (results_subject["target_subject"] == s2)
            ]["no_align"]
            m_score = scores.mean()
            std_score = scores.std()

            dico_distances["mean_score"] = m_score
            dico_distances["std_score"] = std_score

            L_subject["no_align"][s1][s2] = dico_distances

In [21]:
results_mean = np.zeros((len(subjects_src), len(distances)+1))
t_subjects = np.zeros((len(subjects_src), len(distances)))

for l, distance in enumerate(L_subject):
    results = np.zeros((5,5))
    ts = np.zeros((5,5))
    for i, s1 in enumerate(subjects_src):
        for j, s2 in enumerate(subjects_tgt):
            if s1 != s2:
                results[i, j] = L_subject[distance][s1][s2]["mean_score"]
                if distance != "no_align":
                    ts[i, j] = L_subject[distance][s1][s2]["mean_time"]
                
        results_mean[i, l] = np.sum(results[i, :])/4
        
        if distance != "no_align":
            t_subjects[i, l] = np.sum(ts[i, :])/4
                       
    results_pd = pd.DataFrame(results, index=subjects_src, columns=subjects_tgt)
    print(distance)
    print(results_pd)
    print()

les
          1         3         7         8         9
1  0.000000  0.688889  0.597786  0.689394  0.569620
3  0.703297  0.000000  0.719557  0.689394  0.540084
7  0.571429  0.692593  0.000000  0.617424  0.535865
8  0.630037  0.751852  0.542435  0.000000  0.561181
9  0.556777  0.603704  0.394834  0.640152  0.000000

lew
          1         3         7         8         9
1  0.000000  0.700000  0.597786  0.681818  0.535865
3  0.695971  0.000000  0.715867  0.693182  0.548523
7  0.578755  0.703704  0.000000  0.617424  0.535865
8  0.630037  0.722222  0.542435  0.000000  0.556962
9  0.553114  0.603704  0.402214  0.632576  0.000000

spdsw
          1         3         7         8         9
1  0.000000  0.680000  0.590406  0.687879  0.518143
3  0.684249  0.000000  0.710701  0.692424  0.568776
7  0.576557  0.697778  0.000000  0.608333  0.534177
8  0.627106  0.720741  0.538745  0.000000  0.556962
9  0.539194  0.590370  0.401476  0.601515  0.000000

logsw
          1         3         7         8

In [22]:
pd.DataFrame(results_mean, index=subjects_src, columns=np.concatenate([distances, ["no_align"]]))

,les,lew,spdsw,logsw,no_align
1,0.636422,0.628867,0.619107,0.605029,0.420933
3,0.663083,0.663386,0.664038,0.665319,0.356190
7,0.604328,0.608937,0.604211,0.572917,0.395167
8,0.621376,0.612914,0.610889,0.601895,0.429028
9,0.548866,0.547902,0.533139,0.506280,0.299386


In [23]:
pd.DataFrame(results_mean, index=subjects_src, columns=np.concatenate([distances, ["no_align"]])).mean(axis=0)

les         0.614815
lew         0.612401
spdsw       0.606277
logsw       0.590288
no_align    0.380141
dtype: float64

In [24]:
pd.DataFrame(results_mean, index=subjects_src, columns=np.concatenate([distances, ["no_align"]])).std(axis=0)

les         0.042717
lew         0.041961
spdsw       0.047105
logsw       0.057751
no_align    0.053326
dtype: float64

In [25]:
pd.DataFrame(t_subjects, index=subjects_src, columns=distances)

,les,lew,spdsw,logsw
1,11.324229,11.824216,4.383110,4.355354
3,11.384218,11.755733,4.366299,4.345003
7,11.339976,11.736652,4.391261,4.354997
8,11.286586,12.039970,4.350797,4.319424
9,10.723669,11.463459,4.211523,4.186710


In [26]:
pd.DataFrame(t_subjects, index=subjects_src, columns=distances).mean(axis=0)

les      11.211736
lew      11.764006
spdsw     4.340598
logsw     4.312298
dtype: float64

In [27]:
pd.DataFrame(t_subjects, index=subjects_src, columns=distances).std(axis=0)

les      0.275073
lew      0.206716
spdsw    0.073822
logsw    0.071711
dtype: float64